# Importing Libraries

In [218]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision
import torchvision.transforms as transforms
from torchvision.models import (
    resnet50,
    ResNet50_Weights,
    mobilenet_v2,
    MobileNet_V2_Weights,
    vgg16,
    VGG16_Weights,
)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import OneHotEncoder

from PIL import Image


import time

import importlib
import utils

importlib.reload(utils)
from utils import (
    accuracy,
    train,
    combined_train,
    test,
    combined_test,
    plot_acc,
    plot_loss,
    plot_confusion_matrix,
    save_model,
    get_feature_maps,
    visualize_feature_maps,
    plot_images,
    plot_feature_maps,
)


In [219]:
num_classes = 102
torch.manual_seed(13)
np.random.seed(13)

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_size = (224, 224)
batch_size = 32

# Loading Flower102 + Preprocess

In [220]:
train_transform = transforms.Compose(
    [
        transforms.Resize(size=(256, 256)),
        transforms.RandomCrop(size=image_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

val_transform = transforms.Compose(
    [
        transforms.Resize(size=image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std),
    ]
)

In [ ]:
train_dataset = torchvision.datasets.Flowers102(
    root="./data",
    split="test",
    download=True,
    transform=train_transform,
)

val_dataset = torchvision.datasets.Flowers102(
    root="./data",
    split="val",
    download=True,
    transform=val_transform,
)

test_dataset = torchvision.datasets.Flowers102(
    root="./data",
    split="train",
    download=True,
    transform=val_transform,
)

print(f"Train Dataset: {len(train_dataset)}")
print(f"Val Dataset: {len(val_dataset)}")
print(f"Test Dataset: {len(test_dataset)}")

In [222]:
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True
)

In [ ]:
file = open("./Flower102-classes.txt")
classes = np.array([text.strip() for text in file.readlines()])
print(classes)

In [ ]:
unique, counts = np.unique(train_dataset._labels, return_counts=True)
class_weights = 1 / counts
class_weights /= class_weights.sum()
class_weights = torch.tensor(class_weights).float()
print(class_weights)

In [ ]:
images, labels = next(iter(train_loader))
plot_images([images[0]], mean, std, [classes[labels[0]]], "Train Sample")

# Loading MobileNet V2 + Sample Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device.type}")

In [ ]:
test_image = Image.open("test.jpg")
test_image = val_transform(test_image).unsqueeze(0)
test_image.shape
plot_images(test_image, mean, std, ["Test Image"], title="Testing MobileNet V2")

## Testing an Image on MobileNet V2

In [ ]:
mobilenet_model = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
mobilenet_model.classifier

In [ ]:
mobilenet_model.to(device)
test_image = test_image.to(device)
mobilenet_model.eval()

test_output = mobilenet_model(test_image)
top_classes, top_classes_idx = torch.topk(test_output, k=3, dim=1)
top_classes, top_classes_idx = top_classes.squeeze(), top_classes_idx.squeeze()
top_classes, top_classes_idx

In [ ]:
mobilenet_categories = np.array(MobileNet_V2_Weights.DEFAULT.meta["categories"])
top_categories = mobilenet_categories[top_classes_idx.cpu()]
print(f"Top 3 categories: {top_categories}")

## Preparing and Training MobileNet V2 for Classification on Flower102 Dataset

In [231]:
num_features = mobilenet_model.classifier[1].in_features
mobilenet_model.classifier[1] = nn.Linear(
    in_features=num_features, out_features=num_classes, bias=True
)

for param in mobilenet_model.parameters():
    param.requires_grad = False

for param in mobilenet_model.classifier.parameters():
    param.requires_grad = True

In [232]:
mobilenet_model.to(device)
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(mobilenet_model.classifier.parameters())

In [ ]:
start_time = time.time()
epochs = 30

train_losses, train_accuracies, validation_losses, validation_accuracies = train(
    model=mobilenet_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=epochs,
    log=True,
)

end_time = time.time()
print(f"time: {np.round(end_time - start_time, 2)} seconds")

### Plotting Loss and Accuracy

In [ ]:
plot_loss(
    train_losses,
    validation_losses,
    epochs=epochs,
    title="Cross Entropy Loss vs epoch for MobileNet V2",
)
plot_acc(
    train_accuracies,
    validation_accuracies,
    epochs=epochs,
    title="Accuracy vs epoch for MobileNet V2",
)

### Testing MoblieNet V2 on Train, Val, Test Datasets

In [ ]:
train_loss, train_accuracy, train_preds, train_labels = test(
    model=mobilenet_model, test_loader=train_loader, criterion=criterion, device=device
)
print("Train Data:")
print(f"loss= {np.round(train_loss, 3)}, accuracy= {np.round(train_accuracy, 3)}")


val_loss, val_accuracy, val_preds, val_labels = test(
    model=mobilenet_model, test_loader=val_loader, criterion=criterion, device=device
)
print("Val Data:")
print(f"loss= {np.round(val_loss, 3)}, accuracy= {np.round(val_accuracy, 3)}")


test_loss, test_accuracy, test_preds, test_labels = test(
    model=mobilenet_model, test_loader=test_loader, criterion=criterion, device=device
)
print("Test Data:")
print(f"loss= {np.round(test_loss, 3)}, accuracy= {np.round(test_accuracy, 3)}")

In [ ]:
mobilenet_model.eval()

test_output = mobilenet_model(test_image)
top_classes, top_classes_idx = torch.topk(test_output, k=3, dim=1)
top_classes, top_classes_idx = top_classes.squeeze(), top_classes_idx.squeeze()

top_categories = classes[top_classes_idx.cpu()]
print(f"Top 3 categories: {top_categories}")

# Training VGG16

In [ ]:
vgg_model = vgg16(weights=VGG16_Weights.DEFAULT)
vgg_model.classifier

In [238]:
num_features = vgg_model.classifier[6].in_features
vgg_model.classifier[6] = nn.Linear(
    in_features=num_features, out_features=num_classes, bias=True
)

for param in vgg_model.parameters():
    param.requires_grad = False

for param in vgg_model.classifier.parameters():
    param.requires_grad = True

In [239]:
vgg_model.to(device)
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(vgg_model.classifier.parameters())

In [ ]:
start_time = time.time()
epochs = 30

train_losses, train_accuracies, validation_losses, validation_accuracies = train(
    model=vgg_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=epochs,
    log=True,
)

end_time = time.time()
print(f"time: {np.round(end_time - start_time, 2)} seconds")

### Plotting Loss and Accuracy

In [ ]:
plot_loss(
    train_losses,
    validation_losses,
    epochs=epochs,
    title="Cross Entropy Loss vs epoch for VGG16",
)
plot_acc(
    train_accuracies,
    validation_accuracies,
    epochs=epochs,
    title="Accuracy vs epoch for VGG16",
)

### Testing VGG16 on Train, Val, Test Datasets

In [ ]:
train_loss, train_accuracy, train_preds, train_labels = test(
    model=vgg_model, test_loader=train_loader, criterion=criterion, device=device
)
print("Train Data:")
print(f"loss= {np.round(train_loss, 3)}, accuracy= {np.round(train_accuracy, 3)}")

val_loss, val_accuracy, val_preds, val_labels = test(
    model=vgg_model, test_loader=val_loader, criterion=criterion, device=device
)
print("Val Data:")
print(f"loss= {np.round(val_loss, 3)}, accuracy= {np.round(val_accuracy, 3)}")

test_loss, test_accuracy, test_preds, test_labels = test(
    model=vgg_model, test_loader=test_loader, criterion=criterion, device=device
)
print("Test Data:")
print(f"loss= {np.round(test_loss, 3)}, accuracy= {np.round(test_accuracy, 3)}")

In [ ]:
vgg_model.eval()

test_output = vgg_model(test_image)
top_classes, top_classes_idx = torch.topk(test_output, k=3, dim=1)
top_classes, top_classes_idx = top_classes.squeeze(), top_classes_idx.squeeze()

top_categories = classes[top_classes_idx.cpu()]
print(f"Top 3 categories: {top_categories}")

# Training ResNet50

In [ ]:
resnet_model = resnet50(weights=ResNet50_Weights.DEFAULT)
resnet_model.fc

In [245]:
num_features = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(
    in_features=num_features, out_features=num_classes, bias=True
)

for param in resnet_model.parameters():
    param.requires_grad = False

for param in resnet_model.fc.parameters():
    param.requires_grad = True

In [246]:
resnet_model.to(device)
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(resnet_model.fc.parameters())

In [ ]:
start_time = time.time()
epochs = 30

train_losses, train_accuracies, validation_losses, validation_accuracies = train(
    model=resnet_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=epochs,
    log=True,
)

end_time = time.time()
print(f"time: {np.round(end_time - start_time, 2)} seconds")

### Plotting Loss and Accuracy

In [ ]:
plot_loss(
    train_losses,
    validation_losses,
    epochs=epochs,
    title="Cross Entropy Loss vs epoch for ResNet50",
)
plot_acc(
    train_accuracies,
    validation_accuracies,
    epochs=epochs,
    title="Accuracy vs epoch for ResNet50",
)

### Testing ResNet50 on Train, Val, Test Datasets

In [ ]:
train_loss, train_accuracy, train_preds, train_labels = test(
    model=resnet_model, test_loader=train_loader, criterion=criterion, device=device
)
print("Train Data:")
print(f"loss= {np.round(train_loss, 3)}, accuracy= {np.round(train_accuracy, 3)}")

val_loss, val_accuracy, val_preds, val_labels = test(
    model=resnet_model, test_loader=val_loader, criterion=criterion, device=device
)
print("Val Data:")
print(f"loss= {np.round(val_loss, 3)}, accuracy= {np.round(val_accuracy, 3)}")


test_loss, test_accuracy, test_preds, test_labels = test(
    model=resnet_model, test_loader=test_loader, criterion=criterion, device=device
)
print("Test Data:")
print(f"loss= {np.round(test_loss, 3)}, accuracy= {np.round(test_accuracy, 3)}")

In [ ]:
resnet_model.eval()

test_output = resnet_model(test_image)
top_classes, top_classes_idx = torch.topk(test_output, k=3, dim=1)
top_classes, top_classes_idx = top_classes.squeeze(), top_classes_idx.squeeze()

top_categories = classes[top_classes_idx.cpu()]
print(f"Top 3 categories: {top_categories}")

# Training MobileNet V2 with random weights

In [ ]:
mobilenet_random_model = mobilenet_v2(weights=None)
mobilenet_random_model.classifier

In [254]:
num_features = mobilenet_random_model.classifier[1].in_features
mobilenet_random_model.classifier[1] = nn.Linear(
    in_features=num_features, out_features=num_classes, bias=True
)

for param in mobilenet_random_model.parameters():
    param.requires_grad = True

In [255]:
mobilenet_random_model.to(device)
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(mobilenet_random_model.parameters())

In [ ]:
start_time = time.time()
epochs = 30

train_losses, train_accuracies, validation_losses, validation_accuracies = train(
    model=mobilenet_random_model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    device=device,
    epochs=epochs,
    log=True,
)

end_time = time.time()
print(f"time: {np.round(end_time - start_time, 2)} seconds")

### Plotting Loss and Accuracy

In [ ]:
plot_loss(
    train_losses,
    validation_losses,
    epochs=epochs,
    title="Cross Entropy Loss vs epoch for MobileNet V2 (random weights)",
)
plot_acc(
    train_accuracies,
    validation_accuracies,
    epochs=epochs,
    title="Accuracy vs epoch for MobileNet V2 (random weights)",
)

## Testing MoblieNet V2 (random weights) on Train, Val, Test Datasets

In [ ]:
train_loss, train_accuracy, train_preds, train_labels = test(
    model=mobilenet_random_model,
    test_loader=train_loader,
    criterion=criterion,
    device=device,
)
print("Train Data:")
print(f"loss= {np.round(train_loss, 3)}, accuracy= {np.round(train_accuracy, 3)}")


val_loss, val_accuracy, val_preds, val_labels = test(
    model=mobilenet_random_model,
    test_loader=val_loader,
    criterion=criterion,
    device=device,
)
print("Val Data:")
print(f"loss= {np.round(val_loss, 3)}, accuracy= {np.round(val_accuracy, 3)}")


test_loss, test_accuracy, test_preds, test_labels = test(
    model=mobilenet_random_model,
    test_loader=test_loader,
    criterion=criterion,
    device=device,
)
print("Test Data:")
print(f"loss= {np.round(test_loss, 3)}, accuracy= {np.round(test_accuracy, 3)}")

In [ ]:
mobilenet_random_model.eval()

test_output = mobilenet_random_model(test_image)
top_classes, top_classes_idx = torch.topk(test_output, k=3, dim=1)
top_classes, top_classes_idx = top_classes.squeeze(), top_classes_idx.squeeze()

top_categories = classes[top_classes_idx.cpu()]
print(f"Top 3 categories: {top_categories}")